In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on
# Загрузка линтера

In [2]:
import pickle
import sklearn

# Загрузка tf -idf эмбеддинга
vectorizer = None
with open('SVC/vec.pickle', 'rb') as f:
    vectorizer = pickle.load(f)

# Загрузка SVC модели
SVC = None
with open('SVC/svc.pickle', 'rb') as f:
    SVC = pickle.load(f)

## **Чтение pdf и классификация**

In [3]:
from PyPDF2 import PdfReader


# Чтение pdf файла при помощи Pypdf2
def getText(path: str) -> str:
    reader = PdfReader(path)
    number_of_pages = len(reader.pages)

    # Итерация по страницам и добавление текста страницы в
    # в полный тест документа
    fullDocText = ""
    for i in range(number_of_pages):
        page = reader.pages[i]
        fullDocText += page.extract_text()

    return fullDocText

In [4]:
# Файл с небольшим количеством pdf документов с текстами на различную тематику
carPdfPath = "pdfExamples/cars.pdf"

text = getText(carPdfPath)
text

'1 Automotive design\nThe modern automobile is a complex technical system employing subsystems\nwith specific design functions. Some of these consist of thousands of component\nparts that have evolved from breakthroughs in existing technology or from new\ntechnologies such as electronic computers, high-strength plastics, and new alloys\nof steel and nonferrous metals. Some subsystems have come about as a result\nof factors such as air pollution, safety legislation, and competition between\nmanufacturers throughout the world.\nPassenger cars have emerged as the primary means of family transportation,\nwith an estimated 1.4 billion in operation worldwide. About one-quarter of these\nare in the United States, where more than three trillion miles (almost five trillion\nkilometres) are traveled each year. In recent years, Americans have been offered\nhundreds of different models, about half of them from foreign manufacturers. To\ncapitalize on their proprietary technological advances, manuf

## **Импорт пайплайна**

In [14]:
# Функции препроцессинга были вынесены в preprocess.py
from preprocess import preprocess, target_names, compose
import numpy as np


# Классификаия текста
def classifyText(text: str) -> str:

    # Препроцессин и веторизация
    processedText = preprocess(text)
    vectorized = vectorizer.transform([processedText])

    # Классификация - вывод npмассив
    out = SVC.predict(vectorized)
    # Выделение числа - ID класса из массива np.array -> int
    clsId = np.squeeze(out)
    # Получить название класса
    cls = target_names[clsId]

    return cls

In [13]:
classifyText(text)

'rec.autos'

In [17]:
# Пайплайн для классификации файла
classifyPdf = compose(classifyText,
                      getText)

classifyPdf("pdfExamples/Religion.pdf")

'soc.religion.christian'

In [20]:
# Отдельного класса для компютерных наук не было в датасете
classifyPdf("pdfExamples/Comp sci.pdf")

'sci.electronics'